# Fake news Detection

### Importing required library
Here I am going to importing some of the required library, if extra library is required to install It will be install later on.

In [1]:
#!pip install xlwings

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
import os
from xlrd import *
import csv 
import sys
import win32com.client as w3c
from tempfile import NamedTemporaryFile

In [4]:
PATH = 'C:/Users/smsoh/Documents/GitHub/CLEF2021/task3/data/Task3a_Batch_1.xlsx'
PASS = 'fake@2021'

In [5]:
xlapp = w3c.Dispatch("Excel.Application")
xlwb = xlapp.Workbooks.Open(PATH, False, True, None, PASS)

In [6]:
#df = pd.read_csv("data/Task3a_Batch_1.xlsx")
f = NamedTemporaryFile(delete=False, suffix='.csv')
f.close()
os.unlink(f.name)

xlCSVWindows = 0x17
xlwb.SaveAs(Filename=f.name, FileFormat=xlCSVWindows)
df = pd.read_csv(f.name)
df

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false
...,...,...,...,...
432,e01f532a,Rosemary Frei and Patrick Corbett A high-prof...,�No one has died from the coronavirus�,false
433,3c791c29,As the coronavirus pandemic continues to devas...,Why Did 21 Million Phone Numbers Disappear fro...,false
434,fe6a9262,Belgium Health Minister Maggie de Block has pu...,Belgium Health Minister puts ban on non-essent...,false
435,97113f55,"Responding To Backlash, Netflix Clarifies Its ...","Responding To Backlash, Netflix Clarifies Its ...",false


### Inserting fake and real dataset

In [8]:
df.head(5)

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false


In [9]:
df.shape

(437, 4)

In [10]:
df['class'] = df['our rating'].apply(lambda x: 0 if 'true' in x.lower() else ( 1 if 'partially false' in x.lower() else ( 2 if 'false' in x.lower() else 3)))

In [11]:
df.head(10)

,public_id,text,title,our rating,class
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false,2
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false,1
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false,1
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false,2
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false,2
5,c03ed5db,Nation UPDATED 8:23 PM - K A B O O M! Governo...,K A B O O M! Governor and Secretary of State i...,other,3
6,61bd9a69,"November 23, 2019 The U.S. Food and Drug Admi...",FDA Shocking Study: Cells Used In Vaccines Con...,false,2
7,bb1999cc,"Trump confirms this was a bombing, not an acci...","Israel Hits Beirut with Nuclear Missile, Trump...",false,2
8,c1dc1ac6,In a show of anti-American sentiment that will...,Obama�s Daughters Caught on Camera Burning US ...,false,2
9,f2182a54,"FEMA camps, portable human cages, it's all rea...",Fields of human cages discovered in Caruthers ...,false,2


In [12]:
df.columns

Index(['public_id', 'text', 'title', 'our rating', 'class'], dtype='object')

In [13]:
df_t = df.drop([ "our rating","public_id"], axis = 1)

In [14]:
df_t = df_t.drop([ "title"], axis = 1)

In [15]:
df_t.head()

,text,class
0,Distracted driving causes more deaths in Canad...,2
1,Missouri politicians have made statements afte...,1
2,Home Alone 2: Lost in New York is full of viol...,1
3,But things took a turn for the worse when riot...,2
4,It�s no secret that Epstein and Schiff share a...,2


In [16]:
df_t.isnull().sum()


text     0
class    0
dtype: int64

#### Randomly shuffling the dataframe 

In [17]:
df_t.reset_index(inplace = True)
df_t.drop(["index"], axis = 1, inplace = True)

In [18]:
df_t.columns

Index(['text', 'class'], dtype='object')

In [19]:
df_t.head()

,text,class
0,Distracted driving causes more deaths in Canad...,2
1,Missouri politicians have made statements afte...,1
2,Home Alone 2: Lost in New York is full of viol...,1
3,But things took a turn for the worse when riot...,2
4,It�s no secret that Epstein and Schiff share a...,2


#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [20]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [21]:
df_t["text"] = df_t["text"].apply(wordopt)

In [22]:
df_t["title"] = df_t["title"].apply(wordopt)

KeyError: 'title'

In [23]:
df_t.head(10)

,text,class
0,distracted driving causes more deaths in canad...,2
1,missouri politicians have made statements afte...,1
2,home alone lost in new york is full of viole...,1
3,but things took a turn for the worse when riot...,2
4,it s no secret that epstein and schiff share a...,2
5,nation updated pm k a b o o m governor a...,3
6,november the u s food and drug administra...,2
7,trump confirms this was a bombing not an acci...,2
8,in a show of anti american sentiment that will...,2
9,fema camps portable human cages it s all rea...,2


#### Defining dependent and independent variable as x and y

In [24]:
x = df_t["text"]
y = df_t["class"]

#### Splitting the dataset into training set and testing set. 

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

#### Convert text to vectors

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### 1. Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [30]:
pred_lr=LR.predict(xv_test)

In [31]:
LR.score(xv_test, y_test)

0.6818181818181818

In [34]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00        21
           2       0.68      1.00      0.81        75
           3       0.00      0.00      0.00         7

    accuracy                           0.68       110
   macro avg       0.17      0.25      0.20       110
weighted avg       0.46      0.68      0.55       110



### 2. Decision Tree Classification

In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [37]:
pred_dt = DT.predict(xv_test)

In [38]:
DT.score(xv_test, y_test)

0.5454545454545454

In [39]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.28      0.24      0.26        21
           2       0.68      0.73      0.71        75
           3       0.00      0.00      0.00         7

    accuracy                           0.55       110
   macro avg       0.24      0.24      0.24       110
weighted avg       0.52      0.55      0.53       110



### 3. Gradient Boosting Classifier

In [40]:
from sklearn.ensemble import GradientBoostingClassifier

In [41]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [42]:
pred_gbc = GBC.predict(xv_test)

In [43]:
GBC.score(xv_test, y_test)

0.6818181818181818

In [44]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.56      0.24      0.33        21
           2       0.73      0.93      0.82        75
           3       0.00      0.00      0.00         7

    accuracy                           0.68       110
   macro avg       0.32      0.29      0.29       110
weighted avg       0.60      0.68      0.62       110



### 4. Random Forest Classifier

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [47]:
pred_rfc = RFC.predict(xv_test)

In [48]:
RFC.score(xv_test, y_test)

0.6818181818181818

In [49]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00        21
           2       0.68      1.00      0.81        75
           3       0.00      0.00      0.00         7

    accuracy                           0.68       110
   macro avg       0.17      0.25      0.20       110
weighted avg       0.46      0.68      0.55       110

C:\Users\smsoh\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Model Testing With Manual Entry

### News

In [50]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [51]:
news = str(input())
manual_testing(news)



LR Prediction: None 
DT Prediction: None 
GBC Prediction: None 
RFC Prediction: None
